<a href="https://colab.research.google.com/github/dalgual/aidatasci/blob/main/opentable_sentiment_helpful.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install tensorflow

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00


In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
!pip install datasets transformers huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [ ]:
# !apt-get install git-lfs


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
import pandas as pd

data_files = "https://raw.githubusercontent.com/dalgual/aidatasci/master/data/classification/opentable_review.csv"
#pd.options.display.max_rows = 9 #999

df = pd.read_csv(data_files) #'data.csv')

#print(df)

In [ ]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46392 entries, 0 to 46391
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  46392 non-null  int64 
 1   nick        46392 non-null  object
 2   score       46392 non-null  int64 
 3   content     46349 non-null  object
 4   date        46392 non-null  object
 5   helpful     46392 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.1+ MB


In [ ]:
import numpy as np

#define conditions
conditions = [
    df['helpful']==0,
    df['helpful']>0
]

#define results
results = [0, 1]

#create new column based on conditions in column1 and column2
df['labels'] = np.select(conditions, results)


In [ ]:
print(df)

       Unnamed: 0           nick  score  \
0           45435  A Google user      5   
1           45541  A Google user      3   
2           45925  A Google user      5   
3           45446  A Google user      4   
...           ...            ...    ...   
46388         170     Kent Weber      4   
46389        3525  Tim Bolognone      5   
46390         238     Neil Stout      4   
46391       15023    Taeyang Kim      5   

                                                 content                 date  \
0      I just found and reserved a table for dinner t...  2009-09-14 17:21:18   
1      Good start takes up too much space and leaves ...  2009-09-14 21:00:11   
2                                 Koo app . Very useable   2009-09-15 3:28:32   
3      Awesome, just waaaaaayyyyy too big for this ki...  2009-09-15 15:43:21   
...                                                  ...                  ...   
46388  Very useful app when exploring a new area. Jus...  2023-01-15 23:57:52   
46

In [ ]:
from datasets import load_dataset
from datasets import Dataset

# dataset = load_dataset("csv", data_files=data_files, split="train")
dataset = Dataset.from_pandas(df)


In [ ]:
print(dataset)

Dataset({
    features: ['Unnamed: 0', 'nick', 'score', 'content', 'date', 'helpful', 'labels'],
    num_rows: 46392
})


In [ ]:
# https://discuss.huggingface.co/t/how-to-split-hugging-face-dataset-to-train-and-test/20885/3
train_ds=dataset.train_test_split(test_size=0.3, shuffle=True) #, stratify_by_column='Sentiment')
print(train_ds)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'content', 'date', 'helpful', 'labels'],
        num_rows: 32474
    })
    test: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'content', 'date', 'helpful', 'labels'],
        num_rows: 13918
    })
})


In [ ]:
#train_ds = train_ds.rename_column("Sentiment", "labels")
train_ds = train_ds.rename_column("content", "text")
print(train_ds)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels'],
        num_rows: 32474
    })
    test: Dataset({
        features: ['Unnamed: 0', 'nick', 'score', 'text', 'date', 'helpful', 'labels'],
        num_rows: 13918
    })
})


In [ ]:
small_train_dataset = train_ds["train"].shuffle(seed=42).select([i for i in list(range(1352))])
small_test_dataset = train_ds["test"].shuffle(seed=42).select([i for i in list(range(580))])


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [ ]:
def preprocess_function(examples):
   #return tokenizer(examples["SentimentText"], truncation=True)
   return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/1352 [00:00<?, ? examples/s]

Map:   0%|          | 0/580 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
   load_prec = load_metric("precision")
   load_recall = load_metric("recall")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   prec = load_prec.compute(predictions=predictions, references=labels)["precision"]
   recall = load_recall.compute(predictions=predictions, references=labels)["recall"]

   return {"accuracy": accuracy, "f1": f1, "precision":prec, "recall":recall}


In [ ]:
#!pip install accelerate -U

In [ ]:
#!pip install transformers[torch]

In [ ]:
from transformers import TrainingArguments, Trainer


#repo_name = "finetuning-sentiment-model-3000-samples-jwoo5"
repo_name = "semadalg/finetuning-opetable-model"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)




In [ ]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


Step,Training Loss


In [ ]:
trainer.evaluate()

In [ ]:
trainer.push_to_hub()


In [ ]:
from transformers import pipeline

sentiment_model = pipeline(model="semadalg/finetuning-opetable-model")
sentiment_model(["I love this move", "This movie sucks!"])


## References
1. Getting Started with Sentiment Analysis using Python, https://huggingface.co/blog/sentiment-analysis-python
1. The model did not return a loss from the inputs, only the following keys: logits, https://discuss.huggingface.co/t/the-model-did-not-return-a-loss-from-the-inputs-only-the-following-keys-logits-for-reference-the-inputs-it-received-are-input-values/25420/3
